# Week 1 Visualizations: Economic Sectors & Conflict Analysis

**Input**: ACLED events + Economics master (sector percentages)  
**Output**: JSON datasets for D3.js charts in viz-datasets/

In [6]:
## Setup

In [7]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

# For visualizations (optional)
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set style for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Data

In [8]:
raw_data_path = Path('../raw-data')
processed_data_path = Path('../processed-data')

df_acled = pd.read_csv(raw_data_path / 'ACLED' / 'ACLED_2025-10-29.csv', encoding='utf-8-sig')
df_econ = pd.read_csv(processed_data_path / 'economics-countries-master.csv')

print(f"ACLED: {len(df_acled):,} events")
print(f"Economics: {len(df_econ):,} country-years")

ACLED: 2,372,683 events
Economics: 10,936 country-years


## Process & Join

In [ ]:
# Filter to 2015+
df_acled_recent = df_acled[df_acled['year'] >= 2015].copy()

# Aggregate events by country-year
conflict_summary = df_acled_recent.groupby(['country', 'year']).agg({
    'event_id_cnty': 'count',
    'fatalities': 'sum'
}).reset_index()
conflict_summary.columns = ['country', 'year', 'event_count', 'total_fatalities']

# Get latest economics data per country
df_econ_latest = df_econ.sort_values('Year').groupby('Country').last().reset_index()

# Join conflict + economics
df_merged = conflict_summary.merge(
    df_econ_latest, 
    left_on='country', 
    right_on='Country',
    how='left'
)

print(f"Merged: {len(df_merged):,} rows")

Merged: 1,606 rows
       country  year  event_count  total_fatalities      Country    Year  \
0  Afghanistan  2017        13019             36607  Afghanistan  2023.0   
1  Afghanistan  2018        14135             43264  Afghanistan  2023.0   
2  Afghanistan  2019        13908             41608  Afghanistan  2023.0   
3  Afghanistan  2020        10339             31359  Afghanistan  2023.0   
4  Afghanistan  2021         9809             42714  Afghanistan  2023.0   

   Primary_%  Secondary_%  Tertiary_%  Tourism_%       GDP_USD  
0      41.89          8.9       49.21        NaN  1.426650e+10  
1      41.89          8.9       49.21        NaN  1.426650e+10  
2      41.89          8.9       49.21        NaN  1.426650e+10  
3      41.89          8.9       49.21        NaN  1.426650e+10  
4      41.89          8.9       49.21        NaN  1.426650e+10  


## Viz 1: Bar Chart - Top Countries by Primary Sector %

In [10]:
# Aggregate by country
country_totals = df_merged.groupby('country').agg({
    'event_count': 'sum',
    'total_fatalities': 'sum',
    'Primary_%': 'first',
    'Secondary_%': 'first',
    'Tertiary_%': 'first',
    'Tourism_%': 'first'
}).reset_index()

# Filter for countries with economics data
country_totals = country_totals[country_totals['Primary_%'].notna()].copy()

# Top 15 by conflict, sorted by Primary_%
viz1_data = country_totals.nlargest(15, 'event_count').sort_values('Primary_%', ascending=False)

viz1_data.head(10)

,country,event_count,total_fatalities,Primary_%,Secondary_%,Tertiary_%,Tourism_%
195,Somalia,30837,54487,60.86,6.68,32.47,NaN
99,Iraq,59606,106705,48.81,9.21,41.99,NaN
0,Afghanistan,67194,202145,41.89,8.90,49.21,NaN
152,Nigeria,32148,80338,31.10,25.30,43.60,4.29
159,Pakistan,70772,19536,29.49,17.07,53.44,NaN
143,Myanmar,78505,79808,26.55,31.96,41.49,NaN
232,Yemen,93875,162361,25.29,15.16,59.55,NaN
96,India,170146,12189,22.10,22.86,55.04,2.60
48,Colombia,29049,12547,20.48,16.44,63.08,2.35
220,Ukraine,197663,137716,18.74,11.22,70.04,NaN


# Prepare JSON

In [11]:
viz_data = viz1_data[['country', 'event_count', 'total_fatalities', 
                      'Primary_%', 'Secondary_%', 'Tertiary_%', 'Tourism_%']].copy()

# Clean values
viz_data['event_count'] = viz_data['event_count'].astype(int)
viz_data['total_fatalities'] = viz_data['total_fatalities'].astype(int)
viz_data['Primary_%'] = viz_data['Primary_%'].round(2)
viz_data['Secondary_%'] = viz_data['Secondary_%'].round(2)
viz_data['Tertiary_%'] = viz_data['Tertiary_%'].round(2)
viz_data['Tourism_%'] = viz_data['Tourism_%'].fillna(0).round(2)

# Metadata
metadata = {
    'title': 'Top Conflict Countries by Economic Sector (2015-2024)',
    'description': 'Top 15 sorted by primary sector %, showing economic structure vs conflict',
    'source': 'ACLED + World Bank',
    'date_range': '2015-2024'
}

viz_data

,country,event_count,total_fatalities,Primary_%,Secondary_%,Tertiary_%,Tourism_%
195,Somalia,30837,54487,60.86,6.68,32.47,0.00
99,Iraq,59606,106705,48.81,9.21,41.99,0.00
0,Afghanistan,67194,202145,41.89,8.90,49.21,0.00
152,Nigeria,32148,80338,31.10,25.30,43.60,4.29
159,Pakistan,70772,19536,29.49,17.07,53.44,0.00
143,Myanmar,78505,79808,26.55,31.96,41.49,0.00
232,Yemen,93875,162361,25.29,15.16,59.55,0.00
96,India,170146,12189,22.10,22.86,55.04,2.60
48,Colombia,29049,12547,20.48,16.44,63.08,2.35
220,Ukraine,197663,137716,18.74,11.22,70.04,0.00


# Save JSON

In [12]:
viz_datasets_path = Path('../viz-datasets')
viz_datasets_path.mkdir(exist_ok=True)

output = {
    'metadata': metadata,
    'data': viz_data.to_dict('records')
}

output_file = viz_datasets_path / 'viz1_bar_chart_sectors_conflicts.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output, f, indent=2, ensure_ascii=False)

print(f"✓ Saved: {output_file.name} ({output_file.stat().st_size / 1024:.1f} KB)")

✓ Saved: viz1_bar_chart_sectors_conflicts.json (3.3 KB)


## Viz 2: Grouped Bar Chart - Eventtypes grouped by Countries

In [19]:
# Group by country and event_type, count events
events_by_country = df_acled_recent.groupby(['country', 'event_type'])['event_id_cnty'].count().reset_index()

# Pivot the data to get event types as columns
pivot_df = events_by_country.pivot(index='country', columns='event_type', values='event_id_cnty').fillna(0)

# sort by total events
pivot_df['total_events'] = pivot_df.sum(axis=1)
pivot_df = pivot_df.sort_values('total_events', ascending=False).drop(columns=['total_events'])

# filter the first 10 countries
pivot_df = pivot_df.head(10)

# Reset index to make country a column
pivot_df = pivot_df.reset_index()

# Create the JSON structure
output_data = {
    "metadata": {
        "title": "Event Types by Country (2015-2024)",
        "description": "Distribution of ACLED event types across countries",
        "source": "ACLED",
        "date_range": "2015-2024"
    },
    "data": pivot_df.to_dict('records')
}

# Save to JSON file
with open('../viz-datasets/viz2_event_types.json', 'w') as f:
    json.dump(output_data, f, indent=2)

pivot_df.head()
print(f"{len(pivot_df):,} rows")

10 rows


## Additional Visualizations

In [ ]:
# Add cells below for:
# - Heatmap (years × event types)
# - 100% stacked bar (sector composition)